<a href="https://colab.research.google.com/github/staerkjoe/QuantRisk_Colab/blob/main/TicketCancellation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pkdarabi/classification-of-travel-purpose")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'classification-of-travel-purpose' dataset.
Path to dataset files: /kaggle/input/classification-of-travel-purpose


Link to dataset:
https://www.kaggle.com/datasets/pkdarabi/classification-of-travel-purpose

In [ ]:
!pip install unidecode

In [ ]:
import tensorflow as tf
import tf_keras as keras
import pandas as pd
# note: tensorflow 2.17 onwards uses tf.keras 3.0 internally,
# which seem to be still bugged and inconsistent.
# We therefore use tf_keras in according to tensorflow version.

import numpy as np
import os
import time
from unidecode import unidecode

In [ ]:
for file in os.listdir(path):
    print(file)

train_data.csv


In [ ]:
df = pd.read_csv(os.path.join(path, "train_data.csv"))

In [ ]:
# drop columns that are unnecessary/not helpful for prediction

# List of columns to drop
drop_cols = [
    'BillID',               # Unique identifier
    'TicketID',             # Unique identifier
    'UserID',               # Sparse and semi-unique
    'BuyerMobile',          # Personal identifier
    'NationalCode',         # Personal identifier
    'HashPassportNumber_p', # Sparse and hashed
    'HashEmail',            # Sparse and hashed
    'CancelTime',           # Leaks target label
    'VehicleType'
]

# Drop columns from DataFrame
df = df.drop(columns=drop_cols)

# Optional: Confirm result
print("Remaining columns:", df.columns.tolist())




Remaining columns: ['Created', 'DepartureTime', 'ReserveStatus', 'Male', 'Price', 'CouponDiscount', 'From', 'To', 'Domestic', 'VehicleClass', 'TripReason', 'Vehicle', 'Cancel']


In [ ]:
# Drop Nulla values for From and TO
df = df.dropna(subset=['From', 'To'])

# Combine 'From' and 'To' columns to create a list of all cities
all_cities = df['From'].tolist() + df['To'].tolist()

# Create a set of unique cities and convert it back to a list
unique_cities = list(set(all_cities))

# print(unique_cities)

# Mapping erstellen
city_mapping = {city: unidecode(city) for city in unique_cities}

'''
# Ausgabe als Dictionary
print("city_mapping = {")
for k, v in city_mapping.items():
    print(f"    '{k}': '{v}',")
print("}")
'''

'\n# Ausgabe als Dictionary\nprint("city_mapping = {")\nfor k, v in city_mapping.items():\n    print(f"    \'{k}\': \'{v}\',")\nprint("}")\n'

In [ ]:
def normalize_city_name(name):
    if pd.isna(name):
        return None
    return (
        str(name)
        .strip()
        .replace('\u200c', '')  # remove zero-width non-joiner
        .replace(' (', '(')
        .replace(')', ')')
        .replace('‌', '')       # remove zero-width joiner
    )

def safe_unidecode(value):
    if pd.isna(value):
        return ""
    return unidecode(str(value))

df['From'] = df['From'].apply(normalize_city_name).map(city_mapping).apply(safe_unidecode)
df['To'] = df['To'].apply(normalize_city_name).map(city_mapping).apply(safe_unidecode)

df['FromMapped'] = df['From'].isin(city_mapping.values())
df['ToMapped'] = df['To'].isin(city_mapping.values())

# keep only rows where both 'From' and 'To' were mapped
df = df[df["FromMapped"] & df["ToMapped"]].copy()
# drop mapping flags
df.drop(columns=["FromMapped", "ToMapped"], inplace=True)

In [ ]:
df['Price'].describe()

,Price
count,9.167200e+04
mean,3.333467e+06
std,7.696764e+06
min,-1.514000e+06
25%,1.025000e+06
50%,1.680000e+06
75%,3.150000e+06
max,3.837646e+08


In [ ]:
q25 = df['Price'].quantile(0.25)
q75 = df['Price'].quantile(0.75)

df = df[(df['Price'] >= q25) & (df['Price'] <= q75)].copy()



In [ ]:
# Columns Transformations

# Date FOrmat
df[['Created', 'DepartureTime']] = df[['Created', 'DepartureTime']].apply(pd.to_datetime)

# Price to €
exchange_rate = 0.00002027  # 1 IRR = 0.00002027 EUR
df['Price'] = (df['Price'] * exchange_rate).round(2)
df['CouponDiscount'] = (df['CouponDiscount'] * exchange_rate).round(2)


In [ ]:
# Fearure Engineering
df['LeadTime'] = df['DepartureTime'] - df['Created']
df['IsWeekend'] = df['DepartureTime'].dt.weekday >= 5
df['DiscountRatio'] = np.where(df['Price'] > 0, df['CouponDiscount'] / df['Price'], 0)

In [ ]:
# Rename the column
df.rename(columns={'VehicleClass': 'VIPStatus'}, inplace=True)

# Replace NaN values with 'Unknown'
df['VIPStatus'] = df['VIPStatus'].fillna('Unknown')


In [ ]:
df.head()

,Created,DepartureTime,ReserveStatus,Male,Price,CouponDiscount,Domestic,VIPStatus,TripReason,Vehicle,Cancel,LeadTime,IsWeekend,DiscountRatio,FromFreq,ToFreq
2,2022-09-20 17:25:27.250,2022-09-21 11:00:00,3,False,40.54,0.0,1,True,Work,Bus,0,0 days 17:34:32.750000,False,0.0,3394,1311
7,2022-07-28 13:59:44.843,2022-07-28 16:00:00,3,True,50.27,0.0,1,True,Work,Bus,0,0 days 02:00:15.157000,False,0.0,14641,2671
8,2022-10-23 09:57:18.867,2022-10-24 02:00:00,3,True,24.73,0.0,1,True,Work,Bus,0,0 days 16:02:41.133000,False,0.0,14641,3607
9,2022-03-24 22:16:40.203,2022-03-28 13:00:00,3,True,26.35,0.0,1,True,Int,Bus,0,3 days 14:43:19.797000,False,0.0,14641,1327
14,2022-09-13 20:13:39.853,2022-09-30 20:55:00,2,True,23.48,0.0,1,Unknown,Int,Train,0,17 days 00:41:20.147000,False,0.0,1759,15161


In [ ]:
# use of frequency encoding since there is a lot of different cities and other encodings would not make sense
# loss of interpretability though

df['FromFreq'] = df['From'].map(df['From'].value_counts())
df['ToFreq'] = df['To'].map(df['To'].value_counts())

# drop the city columns
df.drop(columns=['From', 'To'], inplace=True)

In [ ]:
df['DiscountRatio'].describe()

,DiscountRatio
count,45901.000000
mean,0.001288
std,0.015095
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.113043


In [ ]:
df['Vehicle'].value_counts()

,count
Vehicle,
Bus,26712
Train,19001
Plane,184
InternationalPlane,4


In [162]:
# Create a copy of the cleaned DataFrame
cleaned_data = df.copy()

In [ ]:
cleaned_data['Cancel'].value_counts()

,count
Cancel,
0,38291
1,7610


In [163]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


scaler = StandardScaler()

categorical_cols = ['TripReason', 'VIPStatus','Vehicle']

encoder = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
], remainder='passthrough')


In [164]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='Cancel')  # Features
y = df['Cancel']               # Target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

In [ ]:
model = LogisticRegression(
    penalty='l2',         # or 'l1', 'elasticnet', 'none'
    solver='liblinear',   # good for small datasets and 'l1' penalty
    max_iter=1000,        # increase if convergence warnings appear
    class_weight='balanced',  # handles class imbalance automatically
    random_state=42       # ensures reproducibility
)

In [ ]:
pipeline = Pipeline([
    ('preprocessing', encoder),
    ('scaling', scaler),
    ('smote', smote),
    ('model', model)
])

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__C': [0.01, 0.1, 1, 10],
    'model__penalty': ['l1','l2'],
    'model__solver': ['liblinear'],
}

# GridSearchCV setup
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,                # 5-fold cross-validation
    scoring='accuracy',  # or use another metric
    n_jobs=-1            # use all available cores
)

In [ ]:
# Fit grid search on your data
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)